# Using the cache

First make sure that the dependencies are installed:

```bash
pip install -r requirements-dev.txt
```

Then run redis using docker compose:

```bash
docker compose -f docker-compose.redis.yml up -d
```

Then run the notebook. 

If you want to view the data in redis, you can use the redis-cli:

```bash
brew install redis
redis-cli -p 6380
```

In [2]:
import fsspec
from redis_block_fsspec_cache import RedisCachingFileSystem

fs = fsspec.filesystem(
    "s3",
    anon=True,
)

cached_fs = RedisCachingFileSystem(
    redis_host="localhost",
    redis_port=6380,
    expiry_time=60,
    fs=fs,
)

Let try to access a single chunk from a remote GRIB file we know about in an s3 bucket:

```json
"gust\/0.0.0": [
    "s3:\/\/noaa-hrrr-bdp-pds\/hrrr.20230927\/conus\/hrrr.t00z.wrfsubhf00.grib2",
    2884409,
    1133540
],
```

In [16]:
url = "s3://noaa-hrrr-bdp-pds/hrrr.20230927/conus/hrrr.t00z.wrfsubhf00.grib2"
offset = 2884409
length = 1133540

In [17]:
%%time

nocache_chunk = fs.read_block(url, offset=offset, length=length)

CPU times: user 42.2 ms, sys: 19.1 ms, total: 61.3 ms
Wall time: 1.23 s


In [18]:
%%time

uncached_chunk = cached_fs.read_block(url, offset=offset, length=length)

CPU times: user 4.85 ms, sys: 5.54 ms, total: 10.4 ms
Wall time: 71.5 ms


In [19]:
%%time

cached_chunk = cached_fs.read_block(url, offset=offset, length=length)

CPU times: user 4.16 ms, sys: 4.97 ms, total: 9.14 ms
Wall time: 60.9 ms


In [21]:
is_correct_length = len(cached_chunk) == length
is_correct_data = cached_chunk == uncached_chunk and cached_chunk == nocache_chunk
print(f"Correct length: {is_correct_length}")
print(f"Correct data: {is_correct_data}")

Correct length: False
Correct data: False
